In [2]:
import pandas as pd
import numpy as np
import math as math
import matplotlib.pyplot as plt
import seaborn as sns
import random
%matplotlib inline 

In [3]:
# to see all the columns when using head() or describe()
pd.set_option('display.max_columns',60)

In [4]:
df = pd.read_csv("kenya_data/diaries_transactions_all.csv", dtype={'account_startclose_balance': str})

FileNotFoundError: File b'kenya_data/diaries_transactions_all.csv' does not exist

## For now I keep only "business and agriculture loans" and "joint liability loans"

In [ ]:
dfb = df[(df["trx_family_code"]=="FRMLN")&((df["trx_type_code"]==2760)|(df["trx_type_code"]==3247))]

# 1. Get rid of loans of with unexplained balance adjustments

In [ ]:
dfb["trx_prx_purpose"].unique()

We insert a new column that is 0 if there is no unexplained balance adjustment for this account and a 1 if there is one. The column in 1 or 0 for all entries that correspond to that account.

In [6]:
dfb.insert(dfb.shape[1], "acc_unexplained_trx", np.zeros(dfb.shape[0]))

This program goes through all unique accounts, checks whether it has an unexplained balance adjustment and if yes sets the variable u to 1. Then it goes through another for-loop and sets the entries of that column 0 for all accounts that have an unexplained balance adjustment.

In [7]:
for acc in dfb["account_ids"].unique():
    u = 0
    for index,row in dfb[dfb["account_ids"]==acc].iterrows():
        if row["trx_prx_purpose"]=="DECREASE--Unexplained balance adjustment" or row["trx_prx_purpose"]=="INCREASE--Unexplained balance adjustment":
            u = 1
    for index,row in dfb[dfb["account_ids"]==acc].iterrows():
        dfb.at[index,"acc_unexplained_trx"] = u

### Let us check if it worked.

In [8]:
s = set()
for acc in dfb[dfb["acc_unexplained_trx"]==0]["account_ids"].unique():
     for i in dfb[dfb["account_ids"]==acc]["trx_prx_purpose"].unique():
            s.add(i)
print(s)

{'4. Any known fees', '2. New borrowing', '3. Payments', '2. New borrowinng (individual portion)', '5. Interest accruing', '1. Starting balance (today)', '6. Closing Balance--End of last DQ', '7. Refund from lender'}


# Let's now only use the loans that have no unexplained balance adjustments.

In [9]:
dfb = dfb[dfb["acc_unexplained_trx"]==0]

# 2. Truncation into chunks with no new borrowing

We first insert a column for the new account numbers. These will be like the old account numbers but with a .1 or .2 for each chunk after a new borrowing.

In [125]:
# this is sometimes necessary if we mess smt up
# del dfb["new_account_ids"]

In [10]:
dfb.insert(df.columns.get_loc("account_ids")+1,"new_account_ids", ["-"]*(dfb.shape[0]))

Let's again look at the columns we have.

In [127]:
dfb.sample(1)

,hh_ids,unique_hhs,first_trx_date_hh,last_trx_date_hh,tot_hh_daysofobs,tot_hh_monthsofobs,interview_designation,int_date,int_month,int_year,int_yr_mo,first_int_date,account_ids,new_account_ids,unique_accnts,m_ids_owner,unique_hm_owner,account_bsheet_desig,account_startclose_balance,account_formal,account_liquid,first_trx_date_acc,last_trx_date_acc,tot_acc_daysofobs,tot_acc_monthsofobs,trx_id,m_ids_trx,trx_date,trx_month,trx_year,trx_yr_mo,trx_dq_round,trx_stdtime_days_hh,trx_stdtime_mnths_hh,trx_stdtime_days_acc,trx_stdtime_mnths_acc,trx_class_code,trx_class_desc,trx_family_code,trx_family_desc,trx_type_code,trx_type_desc,trx_prx_purpose,trx_prx_purpose_fd,trx_fee,trx_bsheet_direction,trx_mode_code,trx_mode_desc,trx_place_incommunity,trx_distance_km,trx_outlet,trx_direction,trx_value_kes,trx_value_usd,ddd_gift,trx_inkind_units,trx_inkind_value_usd,trx_inkind_value_kes,trx_stdtime_mnths_hh_nr,acc_unexplained_trx
398120,KVIHK40,NaN,10sep2012,03oct2013,388,13,04=Diaries Interview,19mar2013,3,2013,2013_03,04sep2012,60134978362300000,-,NaN,60134547419200000,NaN,Liability,NaN,Formal,NaN,02oct2012,03oct2013,366,12,60136369687100000,60134547419200000,08mar2013,3,2013,2013_03,9.0,179,6,157,5,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,2760,Individual Business or Agriculture Loan,3. Payments,Repayment on financial liability,0.0,Decrease,1.0,Cash,1.0,0.0,07=Group meeting,Outflow,5500.0,64.70588,0,NaN,NaN,NaN,6.172414,0.0


In [245]:
acc = random.choice(dfb[dfb["trx_prx_purpose"]=="2. New borrowing"]["account_ids"].unique())

In [246]:
print(acc)

59134942469700000


acc = 59135900709000000

In [247]:
lna = dfb[dfb["account_ids"]==acc]

In [248]:
lna = lna.sort_values("trx_stdtime_days_acc")

In [249]:
lna

,hh_ids,unique_hhs,first_trx_date_hh,last_trx_date_hh,tot_hh_daysofobs,tot_hh_monthsofobs,interview_designation,int_date,int_month,int_year,int_yr_mo,first_int_date,account_ids,new_account_ids,unique_accnts,m_ids_owner,unique_hm_owner,account_bsheet_desig,account_startclose_balance,account_formal,account_liquid,first_trx_date_acc,last_trx_date_acc,tot_acc_daysofobs,tot_acc_monthsofobs,trx_id,m_ids_trx,trx_date,trx_month,trx_year,trx_yr_mo,trx_dq_round,trx_stdtime_days_hh,trx_stdtime_mnths_hh,trx_stdtime_days_acc,trx_stdtime_mnths_acc,trx_class_code,trx_class_desc,trx_family_code,trx_family_desc,trx_type_code,trx_type_desc,trx_prx_purpose,trx_prx_purpose_fd,trx_fee,trx_bsheet_direction,trx_mode_code,trx_mode_desc,trx_place_incommunity,trx_distance_km,trx_outlet,trx_direction,trx_value_kes,trx_value_usd,ddd_gift,trx_inkind_units,trx_inkind_value_usd,trx_inkind_value_kes,trx_stdtime_mnths_hh_nr,acc_unexplained_trx
23025,KVIHC35,NaN,05aug2012,22sep2013,413,14,04=Diaries Interview,09oct2012,10,2012,2012_10,07aug2012,59134942469700000,59134942469700000.1,1.0,59134311227800000,NaN,Liability,Start,Formal,NaN,09oct2012,20sep2013,346,11,59134976561100000,59134311227800000,09oct2012,10,2012,2012_10,1.0,65,2,0,0,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,2760,Individual Business or Agriculture Loan,1. Starting balance (today),NaN,0.0,Increase,18.0,STARTING BALANCE,NaN,NaN,NaN,NaN,108000.0,1270.58800,0,NaN,NaN,NaN,2.241379,0.0
277437,KVIHC35,NaN,05aug2012,22sep2013,413,14,04=Diaries Interview,05nov2012,11,2012,2012_11,07aug2012,59134942469700000,59134942469700000.1,NaN,59134311227800000,NaN,Liability,NaN,Formal,NaN,09oct2012,20sep2013,346,11,59135211421200000,59134311227800000,18oct2012,10,2012,2012_10,2.0,74,2,9,0,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,2760,Individual Business or Agriculture Loan,3. Payments,Repayment on financial liability,0.0,Decrease,1.0,Cash,1.0,0.000000,07=Group meeting,Outflow,12000.0,141.17650,0,NaN,NaN,NaN,2.551724,0.0
149656,KVIHC35,NaN,05aug2012,22sep2013,413,14,04=Diaries Interview,03dec2012,12,2012,2012_12,07aug2012,59134942469700000,59134942469700000.1,NaN,59134311227800000,NaN,Liability,NaN,Formal,NaN,09oct2012,20sep2013,346,11,59135453246700000,59134311227800000,18nov2012,11,2012,2012_11,3.0,105,3,40,1,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,2760,Individual Business or Agriculture Loan,3. Payments,Repayment on financial liability,0.0,Decrease,1.0,Cash,1.0,0.000000,07=Group meeting,Outflow,12000.0,141.17650,0,NaN,NaN,NaN,3.620690,0.0
19200,KVIHC35,NaN,05aug2012,22sep2013,413,14,06=Cleaning interview,21sep2013,9,2013,2013_09,07aug2012,59134942469700000,59134942469700000.1,NaN,59134311227800000,NaN,Liability,NaN,Formal,NaN,09oct2012,20sep2013,346,11,34138710898300000,59134311227800000,15dec2012,12,2012,2012_12,20.0,132,4,67,2,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,2760,Individual Business or Agriculture Loan,5. Interest accruing,NaN,0.0,Increase,16.0,Automatic,NaN,NaN,NaN,NaN,2000.0,23.52941,0,NaN,NaN,NaN,4.551724,0.0
427271,KVIHC35,NaN,05aug2012,22sep2013,413,14,06=Cleaning interview,21sep2013,9,2013,2013_09,07aug2012,59134942469700000,59134942469700000.1,NaN,59134311227800000,NaN,Liability,NaN,Formal,NaN,09oct2012,20sep2013,346,11,34138710797700000,59134311227800000,15dec2012,12,2012,2012_12,20.0,132,4,67,2,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,2760,Individual Business or Agriculture Loan,2. New borrowing,New borrowing on financial liability,0.0,Increase,3.0,"Transfer into bank, SACCO, or MFI account",0.0,2.189854,"06=Direct deposit, payroll deduction, or stand...",Inflow,10000.0,117.64710,0,NaN,NaN,NaN,4.551724,0.0
260728,KVIHC35,NaN,05aug2012,22sep2013,413,14,04=Diaries Interview,07jan2013,1,2013,2013_01,07aug2012,59134942469700000,59134942469700000.1,NaN,59134311227800000,NaN,Liability,NaN,Formal,NaN,09oct2012,20sep2013,34

## Now that I see this account I realize that we actually need to consider that there might be transactions on the same day. So we need to check whether there is a new borrowing on any given day, not any given row.

Let's see how account ids are saved in the dataframe.

In [217]:
a = lna["account_ids"].unique()[0]
print(type(a))
a = a+".1"
print(a)
print('{:.20f}'.format(float(a)))

<class 'str'>
60135893778700000.1
60135893778700000.00000000000000000000


For some reason the program does not account for the .1 in the float. Maybe it cannot handle that many digits? But I guess it is fine if we have the account numbers as strings.

Let's look at all the different purposes there are.

In [218]:
dfb["trx_prx_purpose"].unique()

array(['6. Closing Balance--End of last DQ',
       '1. Starting balance (today)', '5. Interest accruing',
       '2. New borrowinng (individual portion)', '3. Payments',
       '4. Any known fees', '7. Refund from lender', '2. New borrowing'],
      dtype=object)

The purposes we want to use as starting points are '2. New borrowing', '2. New borrowinng (individual portion)', '1. Starting balance (today)'.

In [219]:
# we set the first account number to be the old account number.1
# the subsequent chunks will be called #.2, #.3, etc
i=1
acc = lna["account_ids"].unique()[0]
a = lna["account_ids"].unique()[0]+".1"
for index, row in lna.iterrows():
    if row["trx_prx_purpose"] == "2. New borrowinng (individual portion)" or row["trx_prx_purpose"] == "2. New borrowing":
        i +=1
        a = acc+"."+str(i)
        lna.at[index,"new_account_ids"]=a
    else:
         lna.at[index,"new_account_ids"]=a
        

In [220]:
lna[["account_ids","new_account_ids","trx_stdtime_days_acc","trx_prx_purpose","acc_unexplained_trx"]]

,account_ids,new_account_ids,trx_stdtime_days_acc,trx_prx_purpose,acc_unexplained_trx
260932,60135893778700000,60135893778700000.1,0,3. Payments,0.0
281777,60135893778700000,60135893778700000.2,0,2. New borrowing,0.0
400489,60135893778700000,60135893778700000.2,56,3. Payments,0.0
194824,60135893778700000,60135893778700000.2,119,3. Payments,0.0
275798,60135893778700000,60135893778700000.2,168,3. Payments,0.0
15825,60135893778700000,60135893778700000.2,260,6. Closing Balance--End of last DQ,0.0


In [221]:
lna["new_account_ids"].unique()

array(['60135893778700000.1', '60135893778700000.2'], dtype=object)

The algorithm worked, but we have the problem mentioned above, i.e. the new borrowing is not the first entry of the day.

### Now let's adjust this such that new borrowings are always the first thing on one day, and then run the same program as before.

In [42]:
for acc in dfb["account_ids"].unique():
    i=0
    # we will use "indices" to save the reordered indices of the dataset for each unique loan
    indices = []
    # we will use "index_bor" to save the index of the new borrowing that happens on a given day,
    #whenever there were also other transactions (since we need that index to be first)
    index_bor = 0
    #only keep the rows of that account
    lna = dfb[dfb["account_ids"]==acc]
    # sort the dataset according to time
    lna = lna.sort_values("trx_stdtime_days_acc")
    # for each day in the dataset of the account, we check whether there was 
    # a new borrowingo or starting balance and if yes reorder
    for d in lna["trx_stdtime_days_acc"].unique():
        # if only one entry that day we just add the index to indices
        if lna[lna["trx_stdtime_days_acc"]==d].shape[0]==1:
            indices.append(lna[lna["trx_stdtime_days_acc"]==d].index[0])
        # if there are more than 1 entry that day
        elif lna[lna["trx_stdtime_days_acc"]==d].shape[0]>1:
            # if there are more than one new borrowings or starting balances, we need to check manually
            if lna[(lna["trx_stdtime_days_acc"]==d)&((lna["trx_prx_purpose"]=="2. New borrowing")|(lna["trx_prx_purpose"]=="2. New borrowing (individual portion)")|(lna["trx_prx_purpose"]=="1. Starting balance (today)"))].shape[0]>1:
                print("Check manually: several new borrowings on the same day.")
            #if there is exactly one new borrowing or starting balance that day (here we will need to check for each loan type what they exactly call new borrowings)
            elif lna[(lna["trx_stdtime_days_acc"]==d)&((lna["trx_prx_purpose"]=="2. New borrowing")|(lna["trx_prx_purpose"]=="2. New borrowing (individual portion)")|(lna["trx_prx_purpose"]=="1. Starting balance (today)"))].shape[0]==1:
                #for each type of new borrowing if there is one that day, put that index first into indices
                for kind in ["2. New borrowing","2. New borrowing (individual portion)","1. Starting balance (today)"]:
                    if kind in lna[lna["trx_stdtime_days_acc"]==d]["trx_prx_purpose"].unique():
                        index_bor = lna[(lna["trx_stdtime_days_acc"]==d) & (lna["trx_prx_purpose"]==kind)].index[0]
                        indices.append(index_bor)
                        #then add all other indices of that day
                        for i in lna[lna["trx_stdtime_days_acc"]==d].index.drop(index_bor):
                           indices.append(i)
            # if there are several entries for that day but none of them new borrowings just add all indices of that day            
            else:
                for i in lna[lna["trx_stdtime_days_acc"]==d].index:
                    indices.append(i)
    # here we reindex the lna dataset
    lna = lna.loc[indices,]
    #v will be the index of the chunk we are currently at
    v=1
    # saving the old account number
    acc = lna["account_ids"].unique()[0]
    # a is the new account number for each chunk
    a = lna["account_ids"].unique()[0]+".1"
    # we go through all rows of lna and change the entries of the column "new_account_ids" in the original dataset.
    for index, row in lna.iterrows():
        # if there is a new borrowing in the row, increase v by 1 and set a to acc.v
        if row["trx_prx_purpose"] == "2. New borrowinng (individual portion)" or row["trx_prx_purpose"] == "2. New borrowing":
            v +=1
            a = acc+"."+str(v)
            dfb.at[index,"new_account_ids"]=a
        # otherwise set the new account number in that row to a
        else:
             dfb.at[index,"new_account_ids"]=a

## I now want to check which categories of loans we should add.

### Students loans

In [12]:
dfs = df[(df["trx_family_code"]=="FRMLN")&(df["trx_type_code"]==2759)]

In [13]:
dfs.shape

(2, 58)

In [14]:
dfs

,hh_ids,unique_hhs,first_trx_date_hh,last_trx_date_hh,tot_hh_daysofobs,tot_hh_monthsofobs,interview_designation,int_date,int_month,int_year,int_yr_mo,first_int_date,account_ids,unique_accnts,m_ids_owner,unique_hm_owner,account_bsheet_desig,account_startclose_balance,account_formal,account_liquid,first_trx_date_acc,last_trx_date_acc,tot_acc_daysofobs,tot_acc_monthsofobs,trx_id,m_ids_trx,trx_date,trx_month,trx_year,trx_yr_mo,trx_dq_round,trx_stdtime_days_hh,trx_stdtime_mnths_hh,trx_stdtime_days_acc,trx_stdtime_mnths_acc,trx_class_code,trx_class_desc,trx_family_code,trx_family_desc,trx_type_code,trx_type_desc,trx_prx_purpose,trx_prx_purpose_fd,trx_fee,trx_bsheet_direction,trx_mode_code,trx_mode_desc,trx_place_incommunity,trx_distance_km,trx_outlet,trx_direction,trx_value_kes,trx_value_usd,ddd_gift,trx_inkind_units,trx_inkind_value_usd,trx_inkind_value_kes,trx_stdtime_mnths_hh_nr
23158,KMAKE14,1.0,01oct2012,08oct2013,372,12,03=Q3,01oct2012,10,2012,2012_10,31aug2012,63134761448600000,1.0,63134390408500000,NaN,Liability,Start,Formal,NaN,01oct2012,07oct2013,371,12,63134912326400000,63134390408500000,01oct2012,10,2012,2012_10,1.0,0,0,0,0,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,2759,Student loan,1. Starting balance (today),NaN,0.0,Increase,18.0,STARTING BALANCE,NaN,NaN,NaN,NaN,247399.0,2910.576,0,NaN,NaN,NaN,0.0000
25126,KMAKE14,NaN,01oct2012,08oct2013,372,12,04=Diaries Interview,07oct2013,10,2013,2013_10,31aug2012,63134761448600000,NaN,63134390408500000,NaN,Liability,Close,Formal,NaN,01oct2012,07oct2013,371,12,63138120397500000,63134390408500000,07oct2013,10,2013,2013_10,20.0,371,12,371,12,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,2759,Student loan,6. Closing Balance--End of last DQ,NaN,0.0,NaN,19.0,CLOSING BALANCE,NaN,NaN,NaN,NaN,247399.0,2910.576,0,NaN,NaN,NaN,12.7931


### I feel like we should not keep this one? Students loans can be very different.

## Personal loans

In [29]:
dfp = df[(df["trx_family_code"]=="FRMLN")&(df["trx_type_code"]==2762)]

In [30]:
dfp.shape

(324, 58)

In [31]:
acc = random.choice(dfp["account_ids"].unique())

In [32]:
lna = dfp[dfp["account_ids"]==acc]

In [33]:
lna

,hh_ids,unique_hhs,first_trx_date_hh,last_trx_date_hh,tot_hh_daysofobs,tot_hh_monthsofobs,interview_designation,int_date,int_month,int_year,int_yr_mo,first_int_date,account_ids,unique_accnts,m_ids_owner,unique_hm_owner,account_bsheet_desig,account_startclose_balance,account_formal,account_liquid,first_trx_date_acc,last_trx_date_acc,tot_acc_daysofobs,tot_acc_monthsofobs,trx_id,m_ids_trx,trx_date,trx_month,trx_year,trx_yr_mo,trx_dq_round,trx_stdtime_days_hh,trx_stdtime_mnths_hh,trx_stdtime_days_acc,trx_stdtime_mnths_acc,trx_class_code,trx_class_desc,trx_family_code,trx_family_desc,trx_type_code,trx_type_desc,trx_prx_purpose,trx_prx_purpose_fd,trx_fee,trx_bsheet_direction,trx_mode_code,trx_mode_desc,trx_place_incommunity,trx_distance_km,trx_outlet,trx_direction,trx_value_kes,trx_value_usd,ddd_gift,trx_inkind_units,trx_inkind_value_usd,trx_inkind_value_kes,trx_stdtime_mnths_hh_nr
19561,KELDK17,NaN,31aug2012,01nov2013,427,14,04=Diaries Interview,01oct2013,10,2013,2013_10,06sep2012,84136514094000000,NaN,56134329355600000,NaN,Liability,Close,Formal,NaN,01feb2013,01oct2013,242,8,105138062478800000,56134329355600000,01oct2013,10,2013,2013_10,19.0,396,13,242,8,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,2762,Consumer/ personal loan (not payday loan),6. Closing Balance--End of last DQ,NaN,0.0,NaN,19.0,CLOSING BALANCE,NaN,NaN,NaN,NaN,5200.0,61.176470,0,NaN,NaN,NaN,13.655170
21186,KELDK17,NaN,31aug2012,01nov2013,427,14,04=Diaries Interview,17apr2013,4,2013,2013_04,06sep2012,84136514094000000,NaN,56134329355600000,NaN,Liability,NaN,Formal,NaN,01feb2013,01oct2013,242,8,105136618046900000,56134329355600000,08apr2013,4,2013,2013_04,8.0,220,7,66,2,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,2762,Consumer/ personal loan (not payday loan),3. Payments,Repayment on financial liability,0.0,Decrease,16.0,Automatic,NaN,NaN,"01=Bank, MFI, SACCO, insurance branch",Outflow,2500.0,29.411760,0,NaN,NaN,NaN,7.586207
67603,KELDK17,NaN,31aug2012,01nov2013,427,14,04=Diaries Interview,02aug2013,8,2013,2013_08,06sep2012,84136514094000000,NaN,56134329355600000,NaN,Liability,NaN,Formal,NaN,01feb2013,01oct2013,242,8,105137542080200000,56134329355600000,25jul2013,7,2013,2013_07,15.0,328,11,174,6,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,2762,Consumer/ personal loan (not payday loan),3. Payments,Repayment on financial liability,0.0,Decrease,16.0,Automatic,1.0,0.000000,"01=Bank, MFI, SACCO, insurance branch",Outflow,3000.0,35.294120,0,NaN,NaN,NaN,11.310340
268955,KELDK17,NaN,31aug2012,01nov2013,427,14,04=Diaries Interview,26jun2013,6,2013,2013_06,06sep2012,84136514094000000,NaN,56134329355600000,NaN,Liability,NaN,Formal,NaN,01feb2013,01oct2013,242,8,105140245833000000,56134329355600000,24jun2013,6,2013,2013_06,13.0,297,10,143,4,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,2762,Consumer/ personal loan (not payday loan),3. Payments,Repayment on financial liability,0.0,Decrease,16.0,Automatic,1.0,0.000000,"01=Bank, MFI, SACCO, insurance branch",Outflow,300.0,3.529412,0,NaN,NaN,NaN,10.241380
433058,KELDK17,NaN,31aug2012,01nov2013,427,14,04=Diaries Interview,01oct2013,10,2013,2013_10,06sep2012,84136514094000000,NaN,56134329355600000,NaN,Liability,NaN,Formal,NaN,01feb2013,01oct2013,242,8,105138062467400000,56134329355600000,26sep2013,9,2013,2013_09,19.0,391,13,237,8,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,2762,Consumer/ personal loan (not payday loan),3. Payments,Repayment on financial liability,0.0,Decrease,16.0,Automatic,0.0,8.651168,"01=Bank, MFI, SACCO, insurance branch",Outflow,3000.0,35.294120,0,NaN,NaN,NaN,13.482760
433378,KELDK17,NaN,31aug2012,01nov2013,427,14,04=Diaries Interview,30aug2013,8,2013,2013_08,06sep2012,84136514094000000,NaN,56134329355600000,NaN,Liability,NaN,Formal,NaN,01feb2013,01oct2013,242,8,105137783672100000,56134329355600000,23aug2013,8,2013,2013_08,17.0,357,12,203,7,findev,"Borro

### From the ones I looked at, it seems that we can add the personal loans.

In [27]:
codes = [2760,2762, 3247]

## School Fees loan

In [34]:
dfsc = df[(df["trx_family_code"]=="FRMLN")&(df["trx_type_code"]==2762)]

In [35]:
dfsc.shape

(324, 58)

In [39]:
acc = random.choice(dfsc["account_ids"].unique())

In [40]:
lna = dfsc[dfsc["account_ids"]==acc]

In [41]:
lna

,hh_ids,unique_hhs,first_trx_date_hh,last_trx_date_hh,tot_hh_daysofobs,tot_hh_monthsofobs,interview_designation,int_date,int_month,int_year,int_yr_mo,first_int_date,account_ids,unique_accnts,m_ids_owner,unique_hm_owner,account_bsheet_desig,account_startclose_balance,account_formal,account_liquid,first_trx_date_acc,last_trx_date_acc,tot_acc_daysofobs,tot_acc_monthsofobs,trx_id,m_ids_trx,trx_date,trx_month,trx_year,trx_yr_mo,trx_dq_round,trx_stdtime_days_hh,trx_stdtime_mnths_hh,trx_stdtime_days_acc,trx_stdtime_mnths_acc,trx_class_code,trx_class_desc,trx_family_code,trx_family_desc,trx_type_code,trx_type_desc,trx_prx_purpose,trx_prx_purpose_fd,trx_fee,trx_bsheet_direction,trx_mode_code,trx_mode_desc,trx_place_incommunity,trx_distance_km,trx_outlet,trx_direction,trx_value_kes,trx_value_usd,ddd_gift,trx_inkind_units,trx_inkind_value_usd,trx_inkind_value_kes,trx_stdtime_mnths_hh_nr
5530,KMOMT03,NaN,30sep2012,09oct2013,374,12,04=Diaries Interview,16nov2012,11,2012,2012_11,04sep2012,61134798041100000,NaN,61134338082500000,NaN,Liability,NaN,Formal,NaN,12oct2012,30apr2013,200,6,61135306548000000,61134338082500000,30oct2012,10,2012,2012_10,3.0,30,1,18,0,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,2762,Consumer/ personal loan (not payday loan),3. Payments,Repayment on financial liability,0.0,Decrease,1.0,Cash,NaN,NaN,"06=Direct deposit, payroll deduction, or stand...",Outflow,2300.0,27.058820,0,NaN,NaN,NaN,1.034483
6551,KMOMT03,NaN,30sep2012,09oct2013,374,12,03=Q3,12oct2012,10,2012,2012_10,04sep2012,61134798041100000,1.0,61134338082500000,NaN,Liability,Start,Formal,NaN,12oct2012,30apr2013,200,6,61135002483400000,61134338082500000,12oct2012,10,2012,2012_10,1.0,12,0,0,0,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,2762,Consumer/ personal loan (not payday loan),1. Starting balance (today),NaN,0.0,Increase,18.0,STARTING BALANCE,NaN,NaN,NaN,NaN,60000.0,705.882300,0,NaN,NaN,NaN,0.413793
9510,KMOMT03,NaN,30sep2012,09oct2013,374,12,04=Diaries Interview,16nov2012,11,2012,2012_11,04sep2012,61134798041100000,NaN,61134338082500000,NaN,Liability,NaN,Formal,NaN,12oct2012,30apr2013,200,6,61135306712300000,61134338082500000,31oct2012,10,2012,2012_10,3.0,31,1,19,0,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,2762,Consumer/ personal loan (not payday loan),4. Any known fees,NaN,1.0,Increase,16.0,Automatic,NaN,NaN,NaN,NaN,692.0,8.141176,0,NaN,NaN,NaN,1.068966
438126,KMOMT03,NaN,30sep2012,09oct2013,374,12,04=Diaries Interview,20may2013,5,2013,2013_05,04sep2012,61134798041100000,NaN,61134338082500000,NaN,Liability,NaN,Formal,NaN,12oct2012,30apr2013,200,6,84136928689400000,61134338082500000,31mar2013,3,2013,2013_03,7.0,182,6,170,5,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,2762,Consumer/ personal loan (not payday loan),3. Payments,Repayment on financial liability,0.0,Decrease,13.0,Standing order,0.0,8.085859,"01=Bank, MFI, SACCO, insurance branch",Outflow,3400.0,40.000000,0,NaN,NaN,NaN,6.275862
438205,KMOMT03,NaN,30sep2012,09oct2013,374,12,04=Diaries Interview,20may2013,5,2013,2013_05,04sep2012,61134798041100000,NaN,61134338082500000,NaN,Liability,NaN,Formal,NaN,12oct2012,30apr2013,200,6,84136928687200000,61134338082500000,30apr2013,4,2013,2013_04,7.0,212,7,200,6,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,2762,Consumer/ personal loan (not payday loan),3. Payments,Repayment on financial liability,0.0,Decrease,13.0,Standing order,0.0,8.085859,"01=Bank, MFI, SACCO, insurance branch",Outflow,3400.0,40.000000,0,NaN,NaN,NaN,7.310345
438257,KMOMT03,NaN,30sep2012,09oct2013,374,12,04=Diaries Interview,20may2013,5,2013,2013_05,04sep2012,61134798041100000,NaN,61134338082500000,NaN,Liability,NaN,Formal,NaN,12oct2012,30apr2013,200,6,111139296567500000,61134338082500000,30apr2013,4,2013,2013_04,7.0,212,7,200,6,findev,"Borrowing, lending, savings or insurance media...",FRMLN,Formal loan,2762,Consumer/ personal loa

### I am not sure if we should add school fees loans.

In [27]:
codes = [2760,2762, 3247]